In [12]:
from tensorflow.keras.models import clone_model
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocessor
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_v2_preprocessor

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50V2

from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image


from IPython.display import clear_output


models_dict = {}
vgg16_dict, inception_v3_dict, resnet50_dict = {} ,{} ,{}


print("Loading VGG16")
model = VGG16(weights='imagenet')
model = Model(model.input, model.layers[-2].output)
vgg16_dict["model"] = clone_model(model)
vgg16_dict["preprocesor"] = vgg16_preprocessor
vgg16_dict["target_size"] = model.input_shape[1],model.input_shape[2]


print("Loading Inception v3")
model = InceptionV3(weights='imagenet')
model = Model(model.input, model.layers[-2].output)
inception_v3_dict["model"] = clone_model(model)
inception_v3_dict["preprocesor"] = inception_v3_preprocessor
inception_v3_dict["target_size"] = model.input_shape[1],model.input_shape[2]


print("Loading ResNet 50")
model = ResNet50V2(weights='imagenet')
model = Model(model.input, model.layers[-2].output)
resnet50_dict["model"] = clone_model(model)
resnet50_dict["preprocesor"] = resnet_v2_preprocessor
resnet50_dict["target_size"] = model.input_shape[1],model.input_shape[2]


models_dict["VGG16"] = vgg16_dict
models_dict["InceptionV3"] = inception_v3_dict
models_dict["ResNet50"] = resnet50_dict

clear_output()

In [13]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
from skimage.io import imsave
import uuid

import matplotlib.pyplot as plt

def extract_features(image_path,model_name):
    model_dict = models_dict[model_name]
    model = model_dict["model"]
    preprocessor = model_dict["preprocesor"]
    target_size = model_dict["target_size"]
    img = cv2.resize(cv2.imread(image_path),target_size)

    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocessor(img_data)
    features = model.predict(img_data)
    return features[0]

Using TensorFlow backend.
C:\Users\theac\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [14]:
def get_paths(directory):
    '''
    Devuelve la ruta relativa de las muestras .bmp
    '''
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".bmp"):
                paths.append(os.path.join(root, file))
    return paths

In [15]:
images = get_paths("CASIA-Polar")

In [16]:
for model_name in models_dict:
    print(model_name)

VGG16
InceptionV3
ResNet50


In [17]:
# features con modelo VGG16
feat_16 = extract_features(images[0],"VGG16")
print(feat_16.shape, feat_16[:10])

(4096,) [0.         0.2088416  0.05923092 0.03097348 0.21106344 0.
 0.1226595  0.18816388 0.3052756  0.        ]


In [18]:
# features con modelo InceptionV3
feat_inceptionV3 = extract_features(images[0],"InceptionV3")
print(feat_inceptionV3.shape, feat_inceptionV3[:10])

(2048,) [2.6871136e-04 6.3560874e-05 1.3768205e-05 2.7167361e-04 2.4098862e-04
 1.4771052e-03 0.0000000e+00 1.4703074e-05 9.0110773e-04 3.4654944e-04]


In [19]:
# features con modelo ResNet
feat_r50 = extract_features(images[0],"ResNet50")
print(feat_r50.shape, feat_r50[:10])

(2048,) [0.04331058 0.00026794 0.01890843 0.         0.01809138 0.0034548
 0.03069521 0.00555325 0.01891976 0.06719786]


In [20]:
import pandas as pd

df = pd.read_csv("iris_data.csv", index_col=0) # quitamos columna unnamed
df.rename(columns={'image':'Image','class':'Clase'}, inplace=True)
df.head()

,Image,pupil x_center,pupil y_center,pupil radius,iris x_center,iris y_center,iris radius,Clase
0,Adrianna_1.bmp,182,134,36,177,134,102,Adrianna
1,Adrianna_2.bmp,174,138,38,172,138,38,Adrianna
2,Adrianna_3.bmp,173,120,38,167,120,104,Adrianna
3,Adrianna_4.bmp,183,122,40,185,122,38,Adrianna
4,Adrianna_5.bmp,177,145,38,179,143,36,Adrianna


In [21]:
def register_to_deepfeatures(register):
    """
    Compute all CNN features    
    
    Parameters
    ----------
    register : Series
        Serie containing metadata of the image
   
    Returns
    -------
    results : Series 
        A Pandas Serie contanining all of the features

    """
    img_name = register.Image
    clase = register.Clase # clase   
    
    img_dir = os.path.join("CASIA-Polar",clase)
    img_path = os.path.join(img_dir,img_name)

#     img = imread(img_path)

    basic_values = pd.Series([img_name,clase],["Name","Class"])

    all_features = []
    all_names = []

    for model_name in models_dict.keys():
        features = extract_features(img_path,model_name)
        names = [f"{model_name}_{i}" for i in range(len(features))]
        all_features+=list(features)
        all_names+= names
        results = pd.Series(all_features,all_names)

    return pd.concat((basic_values,results))


In [22]:
example1 = df.iloc[5]
example1

Image             Adrianna_6.bmp
pupil x_center               178
pupil y_center               134
pupil radius                  38
iris x_center                179
iris y_center                132
iris radius                   40
Clase                   Adrianna
Name: 5, dtype: object

In [23]:
results = register_to_deepfeatures(example1)

In [24]:
results

Name             Adrianna_6.bmp
Class                  Adrianna
VGG16_0                       0
VGG16_1                0.221749
VGG16_2               0.0717243
VGG16_3                       0
VGG16_4                0.272094
VGG16_5                       0
VGG16_6                0.132013
VGG16_7                0.161956
VGG16_8                0.403958
VGG16_9                       0
VGG16_10                      0
VGG16_11              0.0345179
VGG16_12             0.00243432
VGG16_13               0.240357
VGG16_14               0.222172
VGG16_15                      0
VGG16_16                      0
VGG16_17             0.00487533
VGG16_18               0.393555
VGG16_19               0.245614
VGG16_20                0.25703
VGG16_21               0.168653
VGG16_22                      0
VGG16_23                      0
VGG16_24              0.0532591
VGG16_25                      0
VGG16_26               0.420347
VGG16_27               0.480835
                      ...      
ResNet50

In [25]:
df_deep = df.apply(register_to_deepfeatures, axis=1)

In [28]:
df_deep

,Name,Class,VGG16_0,VGG16_1,VGG16_2,VGG16_3,VGG16_4,VGG16_5,VGG16_6,VGG16_7,...,ResNet50_2038,ResNet50_2039,ResNet50_2040,ResNet50_2041,ResNet50_2042,ResNet50_2043,ResNet50_2044,ResNet50_2045,ResNet50_2046,ResNet50_2047
0,Adrianna_1.bmp,Adrianna,0.000000,0.208842,0.059231,0.030973,0.211063,0.0,0.122660,0.188164,...,0.030832,0.023897,0.061663,0.037631,0.004940,0.001935,0.000951,0.006243,0.000412,0.0
1,Adrianna_2.bmp,Adrianna,0.029311,0.176550,0.104535,0.000000,0.238652,0.0,0.087496,0.191389,...,0.031282,0.025459,0.059258,0.035751,0.004261,0.001177,0.000356,0.005293,0.000991,0.0
2,Adrianna_3.bmp,Adrianna,0.000000,0.204211,0.057775,0.059391,0.240103,0.0,0.146315,0.115406,...,0.030772,0.024089,0.061375,0.032439,0.004523,0.001096,0.000778,0.006340,0.000579,0.0
3,Adrianna_4.bmp,Adrianna,0.016710,0.147381,0.046892,0.038491,0.181673,0.0,0.095015,0.116575,...,0.025745,0.022661,0.062832,0.034408,0.003409,0.001090,0.000931,0.005533,0.000087,0.0
4,Adrianna_5.bmp,Adrianna,0.000000,0.207518,0.109551,0.020769,0.232519,0.0,0.110120,0.212331,...,0.032848,0.024392,0.066071,0.038856,0.003937,0.000989,0.000840,0.006463,0.000637,0.0
5,Adrianna_6.bmp,Adrianna,0.000000,0.221749,0.071724,0.000000,0.272094,0.0,0.132013,0.161956,...,0.028865,0.025286,0.061498,0.038387,0.002541,0.001493,0.000515,0.007959,0.000301,0.0
6,Adrianna_7.bmp,Adrianna,0.000000,0.230834,0.171492,0.025421,0.247290,0.0,0.084318,0.165293,...,0.033189,0.024960,0.061645,0.035673,0.006300,0.001376,0.001030,0.006958,0.001179,0.0
7,Aharon_1.bmp,Aharon,0.000000,0.201929,0.000000,0.057508,0.236805,0.0,0.000000,0.093881,...,0.031408,0.026037,0.058033,0.031602,0.004052,0.001190,0.000114,0.006506,0.000124,0.0
8,Aharon_2.bmp,Aharon,0.000000,0.255309,0.000000,0.090820,0.237992,0.0,0.000000,0.061477,...,0.032711,0.029452,0.062085,0.033771,0.003073,0.000333,0.000021,0.004929,0.000492,0.0
9,Aharon_3.bmp,Aharon,0.000000,0.242189,0.045379,0.000000,0.229590,0.0,0.000000,0.164049,...,0.030965,0.030102,0.061433,0.037043,0.003879,0.001011,0.000525,0.006416,0.000854,0.0


In [27]:
df_deep.to_csv("iris_deep_features.csv")

In [29]:
import pandas as pd
data = pd.read_csv("iris_deep_features.csv",index_col=0)
data.head()


,Name,Class,VGG16_0,VGG16_1,VGG16_2,VGG16_3,VGG16_4,VGG16_5,VGG16_6,VGG16_7,...,ResNet50_2038,ResNet50_2039,ResNet50_2040,ResNet50_2041,ResNet50_2042,ResNet50_2043,ResNet50_2044,ResNet50_2045,ResNet50_2046,ResNet50_2047
0,Adrianna_1.bmp,Adrianna,0.000000,0.208842,0.059231,0.030973,0.211063,0.0,0.122660,0.188164,...,0.030832,0.023897,0.061663,0.037631,0.004940,0.001935,0.000951,0.006243,0.000412,0.0
1,Adrianna_2.bmp,Adrianna,0.029311,0.176550,0.104535,0.000000,0.238652,0.0,0.087496,0.191389,...,0.031282,0.025459,0.059258,0.035751,0.004261,0.001177,0.000356,0.005293,0.000991,0.0
2,Adrianna_3.bmp,Adrianna,0.000000,0.204211,0.057775,0.059391,0.240103,0.0,0.146315,0.115406,...,0.030772,0.024089,0.061375,0.032439,0.004523,0.001096,0.000778,0.006340,0.000579,0.0
3,Adrianna_4.bmp,Adrianna,0.016710,0.147381,0.046892,0.038491,0.181673,0.0,0.095015,0.116575,...,0.025745,0.022661,0.062832,0.034408,0.003409,0.001090,0.000931,0.005533,0.000087,0.0
4,Adrianna_5.bmp,Adrianna,0.000000,0.207518,0.109551,0.020769,0.232519,0.0,0.110120,0.212331,...,0.032848,0.024392,0.066071,0.038856,0.003937,0.000989,0.000840,0.006463,0.000637,0.0


In [48]:
data.shape

(756, 8194)

In [30]:
import pickle

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline

In [4]:
pretrained_models = ["VGG16","InceptionV3","ResNet50"]

In [57]:
columns = list(data.columns)
# len(columns) -> 8194

In [58]:
y = data.Class.values
# y.shape -> 756

In [10]:
dataset_names = pretrained_models.copy()
dataset_names

['VGG16', 'InceptionV3', 'ResNet50']

In [ ]:
datasets

In [60]:
# atts_names= []
# for pretrained_model in pretrained_models:
#     atts_names.append([col_name for col_name in columns if pretrained_model in col_name])
len(atts_names)

3

In [62]:
# datasets = [(data[atts].values,y) for atts in atts_names]
len(datasets[0][0])

756

In [38]:
'''
Definition of the SVM parameter search
'''
import numpy as np
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid_svm = dict(gamma=gamma_range, C=C_range)
nested_cv = 5

grid_svm = GridSearchCV(SVC(), param_grid=param_grid_svm, cv=nested_cv)


'''
Definition of the MLP parameter search
'''
alpha_range = np.logspace(-5, -1, 5)
hidden_layer_sizes_range=[(50,),(100,),(200,),(500,),(1000,)]

param_grid_mlp = dict(alpha=alpha_range, hidden_layer_sizes=hidden_layer_sizes_range)


grid_mlp = GridSearchCV(MLPClassifier(max_iter=1000,
                                      early_stopping=True), param_grid=param_grid_mlp, cv=nested_cv)

In [64]:
'''
List of classifiers and their names included in the experimental study
'''

cls_names = [
             #"Nearest Neighbors",
             "SVM", 
             "LogisticRegression",
             #"Decision Tree", "Random Forest",
           #  "Gradient Boosting Trees","MLP"
             ]

classifiers = [
    #make_pipeline(StandardScaler(), KNeighborsClassifier(3)),
    #make_pipeline(StandardScaler(), grid_svm),
    make_pipeline(StandardScaler(), SVC(kernel='linear')),
    make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),    
    #DecisionTreeClassifier(random_state=0),
    #RandomForestClassifier(random_state=0, n_estimators=100),
    #GradientBoostingClassifier(random_state=0, n_estimators=100),
    #make_pipeline(StandardScaler(), grid_mlp)
    ]

In [65]:
def cross_validate_preds_model(X, y, model, num_folds):
        '''
        Perform cross validation with a model and a dataset (X and y),
        and returns the predictions to later obtain the measurements 
        you want
        
        Parameters
        ----------
        X: numpy.array
            Dataset (features)
        Y: numpy.array
            Dataset (Target)
        model: scikit_model
            model to be trained
        num_folds: int
            number of folds in the cross validation
        
        Return
        -------
        array 
            array of prediccions obtained using cross_validation
        '''
        print('\t'+str(model)[:20], end=' - ')
        preds = cross_val_predict(model,X,y,cv=num_folds)
        print('OK')
        
        return preds

def run_all_save(num_folds,filename):
    '''
    Perform cross validation with all models and datasets.
        
        
    Parameters
    ----------
    num_folds: int
        number of folds in the cross validation
    filename: string
        name of the file that stores the predictions obtained using crossvalidation
        
    Return
    -------
    
    ''' 
    
    all_preds = {}

    for dataset,dataset_name in zip(datasets, dataset_names):
        print(dataset_name)
        X,y = dataset
        for model,cls_name in zip(classifiers,cls_names):
            print(cls_name)
            preds = cross_validate_preds_model(X, y, model, num_folds)
            all_preds[(dataset_name,cls_name)]=(y,preds)

    all_preds["cls_names"]=cls_names
    all_preds["dataset_names"]=dataset_names

    with open(filename, 'wb') as fp:
         pickle.dump(all_preds, fp)

In [67]:
results_path = "deep_prueba.obj"

run_all_save(7,results_path)

VGG16
SVM
	Pipeline(memory=None - OK
LogisticRegression
	Pipeline(memory=None - 

C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

OK
InceptionV3
SVM
	Pipeline(memory=None - OK
LogisticRegression
	Pipeline(memory=None - 

C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

OK
ResNet50
SVM
	Pipeline(memory=None - OK
LogisticRegression
	Pipeline(memory=None - 

C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\theac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so

OK
